# Texture-Removed

Here we remove the texture of all lesions by scrambling the pixels spatially on the lesion area, as well as on the surrounding skin separately.

In [1]:
import cv2
import os
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def crop_center(img,cropx):
    y,x,c = img.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropx//2    
    return img[starty:starty+cropx, startx:startx+cropx, :]

def shuffle_pixels(arr):
    p = np.random.permutation(np.arange(arr.shape[0]))
    arr = arr[p, :]
    return arr

def remove_textures_separately(img_name, IMG_SIZE, CROP_SIZE, basepath, seg_basepath):
    seg_path = os.path.join(seg_basepath, img_name)
    img_path = os.path.join(basepath, img_name)

    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = crop_center(img, CROP_SIZE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    seg = cv2.imread(seg_path)
    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))
    seg = crop_center(seg, CROP_SIZE)
    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))

    seg = seg // 255

    # Separate inner and outer region pixels
    flat_img = img.reshape((-1, 3))
    flat_seg = seg.reshape((-1, 3))
    flat_seg = flat_seg[:, 0]

    inner_pixels = flat_img[flat_seg.astype(bool), :]
    outer_pixels = flat_img[~flat_seg.astype(bool), :]

    # Shuffle them separately
    inner_pixels = shuffle_pixels(inner_pixels)
    outer_pixels = shuffle_pixels(outer_pixels)

    # Reassemble image
    flat_img[flat_seg.astype(bool), :] = inner_pixels
    flat_img[~flat_seg.astype(bool), :] = outer_pixels

    new_image = flat_img.reshape(img.shape)
    
    return new_image

# Derm7pt

In [2]:
lst_settings = [
    {
        'IMG_SIZE': 256,
        'CROP_SIZE': 200,
    }
]

for setting in lst_settings:
    IMG_SIZE = setting['IMG_SIZE']
    CROP_SIZE = setting['CROP_SIZE']

    DEST_FOLDER = f'Derm7pt/augmentations/textures-removed_IMGSize-{IMG_SIZE}_CropSize-{CROP_SIZE}'

    data = pd.read_csv('Derm7pt/original/meta/meta.csv')
    data = data['derm']

    for img_name in data:

        img = remove_textures_separately(
            img_name, 
            IMG_SIZE=IMG_SIZE, 
            CROP_SIZE=CROP_SIZE,
            basepath='Derm7pt/original/images',
            seg_basepath='Derm7pt/segmentations/images'
            )

        # Save processed image
        basedir = img_name.split('/')[0]
        basedir = os.path.join(DEST_FOLDER, basedir)
        current_filepath = os.path.join(DEST_FOLDER, img_name)
                
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        
        cv2.imwrite(current_filepath, img)



# ISIC

In [3]:
lst_settings = [
    {
        'IMG_SIZE': 256,
        'CROP_SIZE': 200,
    }
]

for setting in lst_settings:
    IMG_SIZE = setting['IMG_SIZE']
    CROP_SIZE = setting['CROP_SIZE']

    DEST_FOLDER = f'ISIC/augmentations/textures-removed_IMGSize-{IMG_SIZE}_CropSize-{CROP_SIZE}'

    data = pd.read_csv('ISIC/csvs/multi-class_filtered.csv')
    data = data['image']

    for img_name in data:

        img_name = '/'.join(img_name.split('/')[6:])

        img = remove_textures_separately(
            img_name, 
            IMG_SIZE=IMG_SIZE, 
            CROP_SIZE=CROP_SIZE,
            basepath='ISIC/original',
            seg_basepath='ISIC/segmentations'
            )

        # Save processed image
        basedir = '/'.join(img_name.split('/')[:-1])
        basedir = os.path.join(DEST_FOLDER, basedir)
        current_filepath = os.path.join(DEST_FOLDER, img_name)
                
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        
        cv2.imwrite(current_filepath, img)

